In [1]:
import gc
import os
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pickle
from sklearn.utils import resample
from collections import Counter
import math
from sklearn.model_selection import train_test_split
import helper
import dataset
import torch
import torch.nn as nn
from resnet1d.net1d import Net1D

In [2]:
data_files = [name for name in os.listdir('./data')]

dfs = []

for i in range(len(data_files)):
    fname = './data/' + data_files[i]
    df = pd.read_csv(fname)
    # df.columns = [x.strip().lstrip() for x in df.columns]

    dfs.append(df)

df = pd.concat(dfs, axis=0, ignore_index=True)

df.shape

(2830743, 79)

In [3]:
# Remove spaces in the front and the end of the column names for better human reading
df.columns = [x.lstrip().strip().replace('�', '-') for x in df.columns]
df.shape

(2830743, 79)

In [4]:
# replace inf values to nan
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df.shape

(2827876, 79)

In [5]:
df_train, df_val = train_test_split(df, stratify=df['Label'], test_size=0.2, random_state=42)

In [6]:
Counter(df_train['Label'])

Counter({'BENIGN': 1817055,
         'DoS Hulk': 184099,
         'PortScan': 127043,
         'DDoS': 102420,
         'DoS GoldenEye': 8234,
         'FTP-Patator': 6348,
         'SSH-Patator': 4717,
         'DoS slowloris': 4637,
         'DoS Slowhttptest': 4399,
         'Bot': 1565,
         'Web Attack � Brute Force': 1206,
         'Web Attack � XSS': 522,
         'Infiltration': 29,
         'Web Attack � Sql Injection': 17,
         'Heartbleed': 9})

In [7]:
from imblearn.under_sampling import RandomUnderSampler

sampling_strategy = dict(Counter(df_train['Label']))
attack_cnt = 0

for k, v in sampling_strategy.items():
    if v < 20000 or k == 'BENIGN': continue

    # when samples count bigger than 10k, truncate in half
    sampling_strategy[k] = v // 2
    attack_cnt += v // 2

sampling_strategy['BENIGN'] = attack_cnt

under_sampler = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)

X_train = df_train.drop(['Label'], axis=1)
y_train = df_train['Label']

X_train, y_train = under_sampler.fit_resample(X_train, y_train)

print(y_train.shape[0])
Counter(y_train)

445243


Counter({'BENIGN': 206780,
         'DoS Hulk': 92049,
         'PortScan': 63521,
         'DDoS': 51210,
         'DoS GoldenEye': 8234,
         'FTP-Patator': 6348,
         'SSH-Patator': 4717,
         'DoS slowloris': 4637,
         'DoS Slowhttptest': 4399,
         'Bot': 1565,
         'Web Attack � Brute Force': 1206,
         'Web Attack � XSS': 522,
         'Infiltration': 29,
         'Web Attack � Sql Injection': 17,
         'Heartbleed': 9})

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

pca__n_components = 20

preprocessor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("normalize", MinMaxScaler()),
    ("PCA", PCA(n_components=pca__n_components))
])

label_encoder = LabelEncoder()

In [9]:

X_train = preprocessor.fit_transform(X_train)

# doing this since we need all labels in case there's any labels not found in test set
label_encoder.fit(df['Label'])

y_train = label_encoder.transform(y_train)

In [10]:
with open('./dist/preprocessor-resnet.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

with open('./dist/labelencoder-resnet.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [11]:
X_smote, _ = helper.smote_sampling(X_train, y_train)
X_gaussion = helper.add_gaussion(X_smote)
X_flip = helper.invert_array(X_train)

X_new = np.dstack([X_smote, X_gaussion, X_flip])
X_new = X_new.transpose((0, 2, 1))

X_new.shape

(445243, 3, 20)

In [12]:
X_train = np.expand_dims(X_train, axis=2).transpose((0, 2, 1))
X_train.shape

(445243, 1, 20)

In [13]:
org_dataset = dataset.NumpyDataset(X_train, y_train)
pos_dataset = dataset.NumpyDataset(X_new, y_train)

In [14]:
# best params found by hyper parameters tunning
best_params = {'embedding_size': 34, 'lr': 0.010848038400629992, 'weight_decay': 0.00012209180832556052}

In [15]:
import time
import warnings
import optuna
from torch.utils.data import DataLoader
from pytorch_metric_learning.losses import SelfSupervisedLoss, NTXentLoss
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from models import Extractor, Projector, ExtractorMLP
from models import ExtractorMLP
from itertools import cycle
from torch.utils.tensorboard import SummaryWriter
start_time = time.perf_counter()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

embedding_size = best_params['embedding_size']

extractor_1d = Extractor(n_features=pca__n_components, n_channels=1, embedding_size=embedding_size).to(device)
extractor_3d = Extractor(n_features=pca__n_components, n_channels=3, embedding_size=embedding_size).to(device)

projector = Projector(embedding_size=embedding_size).to(device)

lr = best_params['lr']
weight_decay = best_params['weight_decay']
batch_size = 1024
epochs = 200

org_dataloader = DataLoader(org_dataset, batch_size=batch_size,)
pos_dataloader = DataLoader(pos_dataset, batch_size=batch_size)

optimizer_1d = torch.optim.Adam(extractor_1d.parameters(), lr=lr, weight_decay=weight_decay)
optimizer_3d = torch.optim.Adam(extractor_3d.parameters(), lr=lr, weight_decay=weight_decay)
optimizer_projector = torch.optim.Adam(projector.parameters(), lr=lr, weight_decay=weight_decay)

loss_fn = NTXentLoss()
loss_fn = SelfSupervisedLoss(loss_fn, symmetric=False).to(device)

writer = SummaryWriter(log_dir='./runs')

cnt = 0
num_epoch2save = 5

for epoch in range(epochs):
    if epoch != 0 and epoch % num_epoch2save == 0:
        helper.store_checkpoint(model=extractor_1d, name='extractor1d', epoch=epoch, prefix='resnet')
        helper.store_checkpoint(model=extractor_3d, name='extractor3d', epoch=epoch, prefix='resnet')
        helper.store_checkpoint(model=projector, name='projector', epoch=epoch, prefix='resnet')

    total_loss = 0
    for item1, item2 in zip(org_dataloader, cycle(pos_dataloader)):
        X_batch, _ = item1
        X_new_batch, _ = item2

        X_batch = X_batch.to(device)
        X_new_batch = X_new_batch.to(device)

        optimizer_1d.zero_grad()
        optimizer_3d.zero_grad()
        optimizer_projector.zero_grad()

        embedding_1d = extractor_1d(X_batch)
        embedding_3d = extractor_3d(X_new_batch)
        projected_1d = projector(embedding_1d)
        projected_3d = projector(embedding_3d)

        loss = loss_fn(projected_1d, projected_3d)
        loss.backward()

        total_loss += loss.item()
    
        optimizer_1d.step()
        optimizer_3d.step()
        optimizer_projector.step()

    cnt += 1

    print(f'[{epoch+1}/{epochs}]: {total_loss}')
    
    writer.add_scalar('Loss/train-Resnet50-prod', total_loss, cnt)
        

[1/200]: 2413.051329135895
[2/200]: 2190.4341123104095
[3/200]: 2037.429221868515
[4/200]: 1893.5873763561249
[5/200]: 1847.801521062851
[6/200]: 1789.5462653636932
[7/200]: 1737.7479269504547
[8/200]: 1713.5767991542816
[9/200]: 1645.9643623828888
[10/200]: 1611.3001580238342
[11/200]: 1600.012222290039
[12/200]: 1573.8818888664246
[13/200]: 1535.1768295764923
[14/200]: 1511.9584772586823
[15/200]: 1502.646087884903
[16/200]: 1491.2352771759033
[17/200]: 1575.1760308742523
[18/200]: 1444.5855979919434
[19/200]: 1458.4291853904724
[20/200]: 1405.556879043579
[21/200]: 1393.0760142803192
[22/200]: 1383.3884732723236
[23/200]: 1372.887379169464
[24/200]: 1344.930363893509
[25/200]: 1345.8906095027924
[26/200]: 1402.7776849269867
[27/200]: 1334.2850422859192
[28/200]: 1358.575767993927
[29/200]: 1416.8756594657898
[30/200]: 1359.919137954712
[31/200]: 1335.1441538333893
[32/200]: 1329.8170101642609
[33/200]: 1343.576076745987
[34/200]: 1284.7523410320282
[35/200]: 1293.5357122421265
[36/2

In [ ]:
with open('./dist/resnet/extractor1d-final.pkl', 'wb') as f:
    pickle.dump(extractor_1d, f)